# SoleXplorer

In [33]:
using Sole, SoleXplorer
using StatsBase, MLJ
using Random, DataFrames

Load NATOPS dataset, composed of time series

In [4]:
X, y = Sole.load_arff_dataset("NATOPS")

(360×24 DataFrame
 Row │ X[Hand tip l]                      Y[Hand tip l]                      Z ⋯
     │ Array…                             Array…                             A ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [-0.519771, -0.52758, -0.531415,…  [-2.14011, -2.18043, -2.18425, -…  [ ⋯
   2 │ [-0.489753, -0.48607, -0.484529,…  [-1.55293, -1.54966, -1.55206, -…  [
   3 │ [-0.521346, -0.518394, -0.522321…  [-1.72326, -1.72407, -1.72326, -…  [
   4 │ [-0.57022, -0.562064, -0.565967,…  [-1.91196, -1.90369, -1.90527, -…  [
   5 │ [-0.624417, -0.626031, -0.625388…  [-1.84287, -1.84026, -1.84688, -…  [ ⋯
   6 │ [-0.502501, -0.502525, -0.499415…  [-2.17556, -2.15613, -2.18516, -…  [
   7 │ [-0.488461, -0.489463, -0.487539…  [-2.17242, -2.18203, -2.18057, -…  [
   8 │ [-0.468105, -0.410602, -0.473909…  [-1.86535, -1.89011, -1.87105, -…  [
  ⋮  │                 ⋮                                  ⋮                    ⋱
 354 │ [-0.500404, -0.

Set Random for reproducibility

In [ ]:
train_seed = 11
rng        = Random.Xoshiro(train_seed)

TaskLocalRNG()

### Simple examples of using SoleXplorer

## Decision Tree

In [68]:
Random.seed!(train_seed)
model = traintest(X, y; models=(; type=:decisiontree, rng=rng))

ModelConfig:
    setup      =SymbolicModelSet(type=MLJDecisionTreeInterface.DecisionTreeClassifier, features=4)
    classifier =DecisionTreeClassifier(max_depth = -1, …)
    rules      =nothing
    accuracy   =nothing


In [53]:
model.mach

trained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = -1, …)
  args: 
    1:	Source @606 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @453 ⏎ AbstractVector{Multiclass{6}}


In [54]:
model.model

▣ ([std(Z[Elbow l])w1] < 0.08596251206097288)
├✔ ([std(Y[Hand tip r])w1] < 1.1566050775773289)
│ ├✔ ([maximum(Y[Elbow r])w1] < 0.212721)
│ │ ├✔ ([maximum(Y[Hand tip r])w1] < 0.3105835)
│ │ │ ├✔ ([maximum(Y[Elbow r])w1] < 0.0018995000000000001)
│ │ │ │ ├✔ ([mean(Y[Hand tip l])w1] < -2.12836987254902)
│ │ │ │ │ ├✔ Not clear
│ │ │ │ │ └✘ All clear
│ │ │ │ └✘ ([std(X[Hand tip r])w1] < 0.7329422471876528)
│ │ │ │   ├✔ Not clear
│ │ │ │   └✘ All clear
│ │ │ └✘ ([mean(Y[Thumb r])w1] < -1.0434569705882355)
│ │ │   ├✔ ([maximum(Y[Elbow r])w1] < 0.0970635)
│ │ │   │ ├✔ All clear
│ │ │   │ └✘ Not clear
│ │ │   └✘ All clear
│ │ └✘ ([maximum(Y[Hand tip r])w1] < 0.7871275)
│ │   ├✔ Not clear
│ │   └✘ ([std(Y[Wrist r])w1] < 0.8482700353263318)
│ │     ├✔ All clear
│ │     └✘ Not clear
│ └✘ ([std(Y[Elbow l])w1] < 0.05585516323564049)
│   ├✔ I have command
│   └✘ ([minimum(Y[Thumb l])w1] < -1.9731075)
│     ├✔ I have command
│     └✘ All clear
└✘ ([std(X[Thumb r])w1] < 0.5878229455602122)
  ├✔ Lock win

La prediction è fatta sul modello originale, non sul modello Sole. Chiedere se è possibile fare la predizione sul modello Sole.

In [70]:
preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.7361111111111112

Non c'è bisongo di definire se l'esperimento è di classificazione o regressione.

## Random Forest

In [73]:
Random.seed!(train_seed)
model = traintest(X, y; models=(type=:randomforest, rng=rng))

ModelConfig:
    setup      =SymbolicModelSet(type=MLJDecisionTreeInterface.RandomForestClassifier, features=4)
    classifier =RandomForestClassifier(max_depth = -1, …)
    rules      =nothing
    accuracy   =nothing


In [74]:
preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.75

## XGBoost

In [24]:
model = traintest(X, y; models=(; type=:xgboost))

┌ Warning: Parity encountered in bestguess! counts (458 elements): Dict{CategoricalArrays.CategoricalValue{String, UInt32}, Int64}("Spread wings" => 13, "Lock wings" => 169, "Not clear" => 34, "I have command" => 169, "Fold wings" => 39, "All clear" => 34), argmax: Lock wings, max: 169 (sum = 458)
└ @ SoleBase /home/paso/Documents/Aclai/Sole/SoleBase.jl/src/machine-learning-utils.jl:93


ModelConfig:
    setup      =SymbolicModelSet(type=MLJXGBoostInterface.XGBoostClassifier, features=4)
    classifier =XGBoostClassifier(test = 1, …)
    rules      =nothing
    accuracy   =nothing


In [25]:
preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.7777777777777778

## Modal DecisionTree

In [ ]:
Random.seed!(train_seed)
model = traintest(X, y; models=(; type=:modaldecisiontree, rng=rng))

model isa SymbolicModel = true


┌ Info: Precomputing logiset...
└ @ SoleData /home/paso/Documents/Aclai/Sole/SoleData.jl/src/utils/autologiset-tools.jl:277


ModelConfig:
    setup      =SymbolicModelSet(type=ModalDecisionTrees.MLJInterface.ModalDecisionTree, features=4)
    classifier =ModalDecisionTree(max_depth = nothing, …)
    rules      =nothing
    accuracy   =nothing


In [27]:
preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.8194444444444444

## Modal RandomForest

In [ ]:
Random.seed!(train_seed)
model = traintest(X, y; models=(; type=:modalrandomforest, rng=rng))

┌ Info: Precomputing logiset...
└ @ SoleData /home/paso/Documents/Aclai/Sole/SoleData.jl/src/utils/autologiset-tools.jl:277
Applying trees... 100%|██████████████████████████████████| Time: 0:00:06
┌ Warning: Parity encountered in bestguess! counts (10 elements): Dict("Not clear" => 5, "All clear" => 5), argmax: Not clear, max: 5 (sum = 10)
└ @ SoleBase /home/paso/Documents/Aclai/Sole/SoleBase.jl/src/machine-learning-utils.jl:93
┌ Warning: Parity encountered in bestguess! counts (10 elements): Dict("Not clear" => 5, "All clear" => 5), argmax: Not clear, max: 5 (sum = 10)
└ @ SoleBase /home/paso/Documents/Aclai/Sole/SoleBase.jl/src/machine-learning-utils.jl:93
┌ Warning: Parity encountered in bestguess! counts (10 elements): Dict("Not clear" => 5, "All clear" => 5), argmax: Not clear, max: 5 (sum = 10)
└ @ SoleBase /home/paso/Documents/Aclai/Sole/SoleBase.jl/src/machine-learning-utils.jl:93
┌ Warning: Parity encountered in bestguess! counts (10 elements): Dict("Not clear" => 5, "All clea

ModelConfig:
    setup      =SymbolicModelSet(type=ModalDecisionTrees.MLJInterface.ModalRandomForest, features=4)
    classifier =ModalRandomForest(sampling_fraction = 0.7, …)
    rules      =nothing
    accuracy   =nothing


In [19]:
preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.875

## Scegliamo DecisionTree da ottimizzare, perchè è il più scarso

In [85]:
Random.seed!(train_seed)
model = traintest(X, y; models=(; type=:decisiontree, rng=rng))
preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.7361111111111112

## Posso modificare i parametri di preparazione del dataset:
tipo, anzicheè usare una sola "finestra" e ridurre il dato ad una singola dimensione,
potrei usare più finestre e ridurre il dato ad una dimensione per ogni finestra.

In [81]:
Random.seed!(train_seed)
model = traintest(X, y; 
    models=(
        type=:decisiontree,
        winparams=(; type=adaptivewindow, nwindows=5),)
    )

preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.875

Potrei anche decidere che feature utilizzare

In [86]:
Random.seed!(train_seed)
model = traintest(X, y; 
    models=(
        type=:decisiontree,
        features=[minimum, maximum, mean, cov, std],)
    )

preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.7638888888888888

Proviamo a unire?

In [92]:
Random.seed!(train_seed)
model = traintest(X, y; 
    models=(
        type=:decisiontree,
        features=[minimum, maximum, mean, cov, std],
        winparams=(; type=adaptivewindow, nwindows=5),)
    )

preds = MLJ.predict(model.mach, model.ds.Xtest)
yhat = MLJ.mode.(preds)
acc = MLJ.accuracy(yhat, model.ds.ytest)

0.8472222222222222